In [1]:
# ==================
# Library
# ==================
import warnings
warnings.simplefilter('ignore')
import math
import pandas as pd
import numpy as np
import lightgbm as lgb
import time
import datetime
import gc
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold,GroupKFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from contextlib import contextmanager
import logging
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import log_loss
import sys
import time
import feather
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import pickle
%matplotlib inline
from sklearn.metrics import average_precision_score

sys.path.append("../src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage
pd.set_option('display.max_columns', 300)

In [2]:
# ==================
# Constant
# ==================
ex = "004"
TRAIN_PATH = "../input/train.csv"
TEST_PATH = "../input/test.csv"
USER_PATH = "../input/user_x_anime.csv"
SUB_PATH = "../input/sample_submission.csv"
SAVE_OOF_PATH = f"../output/exp/ex{ex}_oof.npy"
SAVE_TEST_SUB_PATH = f"../output/exp/ex{ex}_test_sub.csv"
LOGGER_PATH = f"../output/exp/ex_{ex}.txt"

In [3]:

# ===============
# Settings
# ===============

SEED = 0
N_SPLITS = 5
SHUFFLE = True
LGBM_PARAMS = {'num_leaves': 32,
               'min_data_in_leaf': 64,
               'objective': 'regression',
               'max_depth': -1,
               'learning_rate': 0.05,
               "boosting": "gbdt",
               "bagging_freq": 1,
               "bagging_fraction": 0.8,
               "bagging_seed": SEED,
               "verbosity": -1,
              'reg_alpha': 0.1,
              'reg_lambda': 0.3,
              'colsample_bytree': 0.7,
              'metric':"rmse",
              'num_threads':32,
         }

LGBM_FIT_PARAMS = {
    'num_boost_round': 10000,
    'early_stopping_rounds': 200,
    'verbose_eval': 200,
}

load_feature = ["../output/fe/fe001.feather",
                "../output/fe/fe002.feather",
                "../output/fe/fe003.feather",
                "../output/fe/fe004.feather",]

In [4]:
# ====================
# Function
# ====================

def calc_loss(y_true, y_pred):
    return  np.sqrt(mean_squared_error(y_true, y_pred))


@contextmanager
def timer(name):
    t0 = time.time()
    yield 
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')

In [5]:
LOGGER = logging.getLogger()
FORMATTER = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
setup_logger(out_file=LOGGER_PATH)

2021-08-17 10:57:15,508 - INFO - logger set up


<RootLogger root (DEBUG)>

In [6]:
# ====================
# Main
# ====================
train_raw = pd.read_csv(TRAIN_PATH)
y = train_raw["Score"]
df = None
for i in load_feature:
    if df is not None:
        _df = pd.read_feather(i)
        df  = pd.concat([df, _df], axis=1)
    else:
        df = pd.read_feather(i)
train = df.iloc[:len(train_raw)]
test = df.iloc[len(train_raw):].reset_index(drop=True)

In [7]:
y

0       8.78
1       8.06
2       7.90
3       6.38
4       8.17
        ... 
4995    7.23
4996    6.54
4997    6.51
4998    5.92
4999    6.83
Name: Score, Length: 5000, dtype: float64

In [8]:
with timer("lightgbm"):
    kf = KFold(n_splits=N_SPLITS,random_state=SEED, shuffle=SHUFFLE)
    y_oof = np.empty([len(train),])
    y_test = []
    drop_cols = []
    features = list(train.columns)
    features = [i for i in features if i not in drop_cols]
    feature_importances = pd.DataFrame()
    categorical_features = []
    for fold, (train_idx, valid_idx) in enumerate(kf.split(train,y)):
        print('Fold {}'.format(fold + 1))
        with timer(f"fold {fold}"):
            x_train, y_train = train.iloc[train_idx][features], y.iloc[train_idx]
            x_val, y_val =train.iloc[valid_idx][features], y.iloc[valid_idx]
            print("train:",len(x_train))

            y_pred_valid, y_pred_test, valid_loss, importances, best_iter,_ = train_lgbm(
                        x_train, y_train, x_val, y_val,test[features],
                        categorical_features=categorical_features,
                        feature_name=features,
                        fold_id=fold,
                        lgb_params=LGBM_PARAMS,
                        fit_params=LGBM_FIT_PARAMS,
                        loss_func=calc_loss,
                        calc_importances=True
                    )

            y_oof[valid_idx] = y_pred_valid
            score = calc_loss(y[valid_idx], y_pred_valid)
            LOGGER.info(f'Fold{fold}:CV={score}')
            y_test.append(y_pred_test)
            feature_importances = pd.concat([feature_importances, importances], axis=0, sort=False)

    score = calc_loss(y, y_oof)
    np.save(SAVE_OOF_PATH, y_oof)
    LOGGER.info(f'CV={score}')

Fold 1
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.306701	valid_1's rmse: 0.39295
[400]	training's rmse: 0.24768	valid_1's rmse: 0.390941


2021-08-17 10:57:18,779 - INFO - Fold0:CV=0.39041223117546286
2021-08-17 10:57:18,783 - INFO - [fold 0] done in 3 s


Early stopping, best iteration is:
[309]	training's rmse: 0.270135	valid_1's rmse: 0.390412
Fold 2
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.305395	valid_1's rmse: 0.397367
[400]	training's rmse: 0.246329	valid_1's rmse: 0.39219
[600]	training's rmse: 0.208195	valid_1's rmse: 0.391423


2021-08-17 10:57:22,878 - INFO - Fold1:CV=0.3908684393465019
2021-08-17 10:57:22,884 - INFO - [fold 1] done in 4 s


Early stopping, best iteration is:
[482]	training's rmse: 0.228756	valid_1's rmse: 0.390868
Fold 3
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.301697	valid_1's rmse: 0.426272
[400]	training's rmse: 0.243748	valid_1's rmse: 0.420584


2021-08-17 10:57:26,388 - INFO - Fold2:CV=0.42012038070028535
2021-08-17 10:57:26,397 - INFO - [fold 2] done in 4 s


Early stopping, best iteration is:
[383]	training's rmse: 0.247537	valid_1's rmse: 0.42012
Fold 4
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.306094	valid_1's rmse: 0.403657
[400]	training's rmse: 0.247764	valid_1's rmse: 0.400113
[600]	training's rmse: 0.209511	valid_1's rmse: 0.401525
Early stopping, best iteration is:
[459]	training's rmse: 0.235065	valid_1's rmse: 0.399066


2021-08-17 10:57:30,663 - INFO - Fold3:CV=0.3990656128729114
2021-08-17 10:57:30,676 - INFO - [fold 3] done in 4 s


Fold 5
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.30141	valid_1's rmse: 0.42217
[400]	training's rmse: 0.244549	valid_1's rmse: 0.413556
[600]	training's rmse: 0.206989	valid_1's rmse: 0.413168
Early stopping, best iteration is:
[508]	training's rmse: 0.222663	valid_1's rmse: 0.41286


2021-08-17 10:57:35,430 - INFO - Fold4:CV=0.4128595697516114
2021-08-17 10:57:35,441 - INFO - [fold 4] done in 5 s
2021-08-17 10:57:35,453 - INFO - CV=0.40284174786668864
2021-08-17 10:57:35,455 - INFO - [lightgbm] done in 20 s


In [9]:
y_test_sub =  np.mean(y_test,axis=0)
sub = pd.read_csv(SUB_PATH)

In [10]:
sub["Score"] = y_test_sub
sub.to_csv(SAVE_TEST_SUB_PATH,index=False)

In [11]:
sub

,MAL_ID,Score
0,5,8.254931
1,6,8.259462
2,7,7.510611
3,8,6.605669
4,15,7.502895
...,...,...
7416,47307,6.924865
7417,47398,7.480943
7418,47402,7.237551
7419,47616,7.080706
